In [36]:
import os
import openai
from dotenv import load_dotenv
from llama_index.core import Document
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core import load_index_from_storage
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.indices.postprocessor import SentenceTransformerRerank

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")


documents = SimpleDirectoryReader(input_files = ["introduction-to-natural-language-processing.pdf"]).load_data()
document = Document(text="\n\n".join([doc.text for doc in documents]))

def get_sentence_window_index(document, index_dir, sentence_window_size=3):
    Node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_sentence",
    )

    Settings.llm = OpenAI()
    Settings.embed_model = "local:BAAI/bge-small-en-v1.5"
    Settings.node_parser = Node_parser

    if not os.path.exists(index_dir):
        sentence_index = VectorStoreIndex.from_documents([document])
        sentence_index.storage_context.persist(persist_dir=index_dir)
        
    else:
        sentence_index = load_index_from_storage(StorageContext.from_defaults(persist_dir=index_dir))
    return sentence_index

def get_sentence_window_engine(sentence_index):
    
    postprocessor = MetadataReplacementPostProcessor(target_metadata_key="window",)
    rerank = SentenceTransformerRerank(top_n=2, model="BAAI/bge-reranker-base") 
    sentence_window_engine = sentence_index.as_query_engine(similarity_top_k=6, node_postprocessors=[postprocessor, rerank])
    
    return sentence_window_engine


In [37]:
index_dir = "./sentence_index_1"
sw_index_1 = get_sentence_window_index(document, index_dir, sentence_window_size=1)
sw_engine_1 = get_sentence_window_engine(sw_index_1)

In [38]:
window_response_1 = sw_engine_1.query(
    "How are corpora utilized in grammatical research, and what advancements have been made with the use of computational tools?"
)
window_response_1.response

'Corpora are used in grammatical research to provide data for analysis alongside linguistic research and subjective intuitions of language experts. Recent advancements in computational power and tools have enabled researchers to conduct quantitative studies on various aspects of grammar, such as the frequency of different clause types in English. These tools have also facilitated the testing of predictions made by formal grammars developed within the generative school of linguistics.'

In [39]:
from trulens_eval import Tru
from trulens_eval import TruLlama
from trulens_eval.feedback.provider import OpenAI
from trulens_eval import Feedback
from trulens_eval.app import App
import numpy as np

tru = Tru()

# Initialize provider class
provider = OpenAI()

# Select context to be used in feedback. The location of context is app specific.

def get_evaluation_response(rag_engine, app_id, eval_questions):
    
    context = App.select_context(rag_engine)

    # Define a groundedness feedback function
    f_groundedness = (
        Feedback(provider.groundedness_measure_with_cot_reasons, name="Groundedness")
        .on(context.collect())  # Collect context chunks into a list
        .on_output()
    )

    # Question/answer relevance between overall question and answer.
    f_answer_relevance = (
        Feedback(provider.relevance, name="Answer Relevance")
        .on_input_output()
    )

    # Question/statement relevance between question and each context chunk.
    f_context_relevance = (
        Feedback(provider.context_relevance_with_cot_reasons, name = "Context Relevance")
        .on_input()
        .on(context)
        .aggregate(np.mean)
    )



    # Initialize the recorder
    tru_query_engine_recorder = TruLlama(
        rag_engine,
        app_id= app_id,
        feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance])
    
    for question in eval_questions:
        with tru_query_engine_recorder as recording:
            response = rag_engine.query(question)
    records = recording.get()
    
    return records


In [40]:
eval_questions = [
    "Who introduced the notions of finite-state machines and context-free grammar (CFG) to linguistics?",
    "How did linguists test formal rules of grammar according to Chomsky’s approach?",
    "What has contributed to making the vision of computers understanding ordinary language and holding conversations with humans more plausible in the 21st century?",
    "Why is it often necessary to assign a part of speech (POS) to a word based on its function in context rather than its inherent meaning?",
    "What are the three fundamental concepts in regular expressions (REs) that are also characteristic of finite-state machines?",
    "What is the difference between right-linear and left-linear grammars in terms of finite-state machines?",
    "Why is center-embedding significant in grammars, and what does it allow?",
    "What is a corpus in the context of natural language processing, and what are the three broad categories of corpora?",
    "How are corpora used in modern lexicography, and how do they influence dictionary entries?",
    "How are corpora utilized in grammatical research, and what advancements have been made with the use of computational tools?"
]

records = get_evaluation_response(
    sw_engine_1,
    app_id='sentence window engine 1',
    eval_questions = eval_questions
)

display(records)
tru.run_dashboard()

✅ In Groundedness, input source will be set to __record__.calls[-1].rets.source_nodes[:].node.text.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.calls[-1].rets.source_nodes[:].node.text .


Record(record_id='record_hash_ab743fbe367b007b0d7c7c79d54cb14f', app_id='app_hash_b180f7aa6c6eb4d319cb046c9d449774', cost=Cost(n_requests=1, n_successful_requests=1, n_completion_requests=1, n_classification_requests=0, n_classes=0, n_embedding_requests=0, n_embeddings=0, n_tokens=1057, n_stream_chunks=0, n_prompt_tokens=977, n_completion_tokens=80, n_cortex_guardrails_tokens=0, cost=0.0016255, cost_currency='USD'), perf=Perf(start_time=datetime.datetime(2024, 10, 24, 12, 48, 58, 590463), end_time=datetime.datetime(2024, 10, 24, 12, 49, 6, 141083)), ts=datetime.datetime(2024, 10, 24, 12, 49, 6, 144086), tags='-', meta=None, main_input='How are corpora utilized in grammatical research, and what advancements have been made with the use of computational tools?', main_output='Corpora are used in grammatical research to provide data for analysis alongside linguistic research and subjective intuitions of language experts. Recent advancements in computational power and tools have enabled rese

Starting dashboard ...
Dashboard already running at path:   Network URL: http://192.168.1.2:50631



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [42]:
from trulens.core import TruSession

session = TruSession()
session.reset_database()

Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]
